In [35]:
HOME_DIR = "/home_remote"
HOME = "/home/thi.tra.my.nguyen"

import pandas as pd
import glob
import re
import nltk
from nltk.tokenize import RegexpTokenizer
import zipfile
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import numpy as np
# from gensim.models.fasttext import FastText
# from gensim.utils import tokenize
# from gensim.models.fasttext import load_facebook_model
# from gensim.models.fasttext import load_facebook_vectors
# from gensim import utils
import fasttext
import fasttext.util


import ast
import os

### Word Embedding

Using Wiki pretrained model

In [34]:
import fasttext.util

ft = fasttext.load_model('/home_remote/fastText/cc.en.300.bin')

PermissionError: [Errno 13] Permission denied: 'cc.en.300.bin.gz.part'

In [32]:
os.getcwd()

'/'